# M/M/s/$\infty$ queue problems

In [1]:
from scipy import math
import pandas as pd
import numpy as np

An M/M/s system is a queuing process having Poisson arrival pattern, $s$ servers with $s$ i.i.d expeonential service times.  Service times do not depend on the state of the system.  The system (i.e. queue + service) has infinite capacity and a FIFO queue discipline.

## Hospital Pharmacy example

During the afternoon, a pharmacy based in a large hospital has 2 trained pharmacists on duty to check and fullfill patient prescriptions for drugs to take home with them at discharge. They are are able to handle 30 transactions per hour. The service times are exponentially distributed. During this busy period, prescriptions arrive at the pharmacy according to a Possion process, at a mean rate of 50 per hour.  

**Questions**

1. What is the probability that there are more than 3 prescriptions in the pharmacy at any one time
2. Expected number of drug prescriptions waiting to be fulfilled
3. Expected number of drug prescriptions in the system
4. Expected prescription turnaround time

### Example Solution:

This is a M/M/2 system with $\lambda=50$ and $\mu = 30$

#### Is the system in control?

Let's first check that steady state conditions hold by calculating the traffic intensity $\rho$.

\begin{equation}
\rho = \frac{\lambda}{s\mu}
\label{eq:rho} \tag{1}
\end{equation}

Steady state conditions hold if $\rho < 1$

In [2]:
def traffic_intensity(_lambda, mu, s):
    '''
    calculate the traffic intensity (server utilisation)
    of an M/M/s queue
    '''
    return _lambda / (s * mu)

In [78]:
#calculate traffic intensity
LAMBDA = 40
MU = 15
S = 3

rho = traffic_intensity(LAMBDA, S, MU)
rho

0.8888888888888888

**Conclusion**: $\rho < 1$ steady state conditions will hold.

### 1. Calculate the probability 3 drug orders in the pharmacy at any one time

Steady state probabilities are given by

\begin{equation*}
P_0 = \left[ \sum_{n=0}^{s-1} \frac{\left(\lambda/ \mu \right)^n}{n!} + \frac{\left( \lambda / \mu \right)^s}{s!\left(1-\rho\right)}  \right]^{-1}
\label{eq:p0} \tag{2}
\end{equation*}


\begin{equation}
  P_n = \left\{
    \begin{array}{l}
      \dfrac{\left( \lambda / \mu \right)^n}{n!}p_0,   \>\>\>\>  n \leq s\\ \\
      \dfrac{\left( \lambda / \mu \right)^n}{s!s^{n-s}}p_0,  \>\>\>\> n > s
    \end{array}
  \right.
\label{eq:pn} \tag{3} 
\end{equation}

In [79]:
def prob_system_empty(_lambda, mu, s):
    '''
    The probability that a M/M/s/infinity queue is empty
    '''
    p0 = 0.0
    rho = traffic_intensity(_lambda, mu, s)
    
    for n in range(s):
        p0 += ((_lambda / mu) ** n) / math.factorial(n)

    p0 += ((_lambda / mu) ** s) / (math.factorial(s) * (1 - rho))
    return p0**-1

In [80]:
p0 = prob_system_empty(LAMBDA, MU, S)
print(f'p0 = {p0:.2f}')

p0 = 0.03


In [81]:
def prob_n_in_system(n, _lambda, mu, s, return_all_solutions=True):
    '''
    Calculate the probability that n customers
    in the system (queuing + service)
    
    Parameters:
    --------
    n: int,
        Number of customers in the system
    
    _lambda: float
        Mean arrival rate to system
        
    mu: float
        Mean service rate
        
    s: int
        number of servers
        
    return_all_solutions: bool, optional (default=True)
        Returns all solutions for 0,1 ... n
        
    Returns:
    ------
        np.ndarray of solutions
    
    '''
    p0 = prob_system_empty(_lambda, mu, s)
    probs = [p0]
    
    #for n <= s
    for i in range(1, min(s+1, n+1)):
        pn = (((_lambda / mu)**i) / math.factorial(i)) * p0
        probs.append(pn)
        
    #for n > s
    for i in range(s+1, n+1):
        pn = (((_lambda / mu)**i) / (math.factorial(s) * (s**(n-s)))) * p0
        probs.append(pn)
    
    if return_all_solutions:
        return np.array(probs)
    else:
        return probs[:-1]

In [82]:
prob = prob_n_in_system(3, LAMBDA, MU, S)

#returns: [p0, p1, p2, p3] => probabilities of 3 or less drug orders
prob

array([0.02803738, 0.07476636, 0.09968847, 0.08861198])

In [83]:
#prob.sum() => p(X <=3)
more_than_three = 1 - prob.sum()
print(f'P(X > 3) = {more_than_three:.2f}')

P(X > 3) = 0.71


### 2. Expected number of drug prescriptions waiting to be fullfilled


$L_q$ = Expected number of customers in the queue for service

\begin{equation}
L_q = \frac{p_0\left(\lambda / \mu \right)^s \rho}{s!\left(1 - \rho\right)}
\tag{4}
\end{equation}

In [84]:
def mean_queue_length(_lambda, mu, s):
    '''
    Mean length of queue Lq
    '''
    p0 = prob_system_empty(_lambda, mu, s)
    rho = traffic_intensity(_lambda, mu, s)
    
    lq = (p0 * ((_lambda / mu)**s) * rho) / (math.factorial(s) * (1 - rho))
    return lq

In [85]:
lq = mean_queue_length(LAMBDA, MU, S)

In [86]:
print(f'lq = {lq:.2f}')

lq = 0.71


### 3. Expected number of drug prescriptions in the system

$L_s$ = Expected number of customers in the queue

We have already calculated $L_q$ therefore we will use

\begin{equation}
L_s = L_q + \frac{\lambda}{\mu}
\tag{5}
\end{equation}

In [87]:
ls = lq + (LAMBDA / MU)

In [88]:
print(f'Ls = {ls:.2f}')

Ls = 3.38


### 4. Expected prescription turnaround time

Using:

\begin{equation}
L_s = \lambda W_s = L_q
\tag{5}
\end{equation}

In [89]:
ws = ls / LAMBDA

In [90]:
print(f'Ws = {ws:.2f}')

Ws = 0.08
